In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import math
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from sklearn import preprocessing, model_selection
from sklearn.linear_model import LinearRegression

In [3]:
merged_path = "C:/Users/ozgur/Desktop/dev/Fantasy-Basketball-Player-Recomendation-System/ML/main_merged6.csv"
merged_df = pd.read_csv(merged_path, index_col=None, header=0)
merged_df.drop(['Pos', 'Date', 'STL_AVG', 'BLK Last Week',
       'USG_perc', 'DRtg', 'ORtg', 'AST_perc', 'DRB_perc', 'ORB_perc', 'Starter', 'MP Last Month',
       'BLK_perc', 'TOV_perc', 'STL_perc', 'eFG_perc', 'MP', 'FG', 'FGA', 'MP Last Week', 'PTS_AVG Last Week',
               'FG_perc', '3P', '3PA', '3P_perc', 'FT', 'FTA', 'BLK Last Month', 'TRB_AVG', 'PTS_AVG', '3PA_AVG',
       'FT_perc', 'TRB', 'STL', 'BLK', 'TOV', 'PF', 'W', 'BLK_AVG', 'FGA_AVG', 'FG_perc_AVG', 'FG_AVG Last Week',
       'FGA_AVG Last Week', 'FG_perc_AVG Last Week', 'FG_AVG Last Month', 'ORtg Last Month', 'TOV_AVG', 
       'FGA_AVG Last Month', 'FG_perc_AVG Last Month', 'AST Last Month', 'AST Last Week', 'AST_AVG', 'AST'], axis=1, inplace=True)
print(merged_df.columns)
print(merged_df.dtypes)
print(merged_df.shape)

Index(['Name', 'Team', 'Home', 'PTS', 'PG', 'SG', 'F', 'C',
       'PTS_AVG Last Month', 'MP_AVG', 'FG_AVG', 'Opp', 'ORtg_AVG',
       'eFG_perc_AVG', 'DRtg_AVG', 'USG_perc_AVG', '3P_AVG'],
      dtype='object')
Name                   object
Team                   object
Home                    int64
PTS                     int64
PG                      int64
SG                      int64
F                       int64
C                       int64
PTS_AVG Last Month    float64
MP_AVG                float64
FG_AVG                float64
Opp                    object
ORtg_AVG              float64
eFG_perc_AVG          float64
DRtg_AVG              float64
USG_perc_AVG          float64
3P_AVG                float64
dtype: object
(28911, 17)


In [4]:
merged_df['Team'] = pd.Categorical(merged_df['Team'])
merged_df['Opp'] = pd.Categorical(merged_df['Opp'])
dummies_team = pd.get_dummies(merged_df['Team'], prefix = 'Team')
dummies_opp = pd.get_dummies(merged_df['Opp'], prefix = 'Opp')
merged_df = pd.concat([merged_df, dummies_team], axis=1)
merged_df = pd.concat([merged_df, dummies_opp], axis=1)

In [ ]:
merged_df['Name'] = pd.Categorical(merged_df['Name'])
dummies_name = pd.get_dummies(merged_df['Name'], prefix = '')
merged_df = pd.concat([merged_df, dummies_name], axis=1)
merged_df

In [46]:
train = merged_df.loc[1609:15000]
test = merged_df.loc[15000:15500]
X_train = train.drop(['PTS', 'Name', 'Team', 'Opp'], 1)
y_train = train[['PTS']]
X_test = test.drop(['PTS', 'Name', 'Team', 'Opp'], 1)
y_test = test[['PTS']]

In [48]:
lm = LinearRegression()
lm.fit(X_train, y_train)
y_pred = lm.predict(X_test)
print(mean_squared_error(y_test, y_pred), r2_score(y_test, y_pred))

32.34014139350684 0.6239916989620613


In [23]:
bst = xgb.XGBRegressor(alpha=1, max_depth=10, n_estimators=60, learning_rate=0.1, gamma=10)
bst.fit(X_train, y_train)
y_pred = bst.predict(X_test)
print(mean_squared_error(y_test, y_pred), r2_score(y_test, y_pred))

[13:26:09] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
33.33842813906732 0.6123849437973404


In [24]:
max_depth = [2, 5, 7, 10, 12]
n_estimators = [30, 40, 50, 60, 70]
learning_rate = [0.05, 0.1]
gamma = [0, 1, 10, 30, 50]
alpha = [0, 1, 10, 30, 50]

param_grid = {'max_depth': max_depth,
               'n_estimators': n_estimators,
               'learning_rate': learning_rate,
              'gamma': gamma,
               'alpha': alpha}

bst = xgb.XGBRegressor()
cv_bst = model_selection.GridSearchCV(estimator=bst, param_grid=param_grid, cv = 3)
cv_bst.fit(X_train, y_train)

[13:31:04] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:31:05] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:31:05] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:31:06] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:31:06] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:31:07] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:31:08] WARNI

[13:32:19] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:32:21] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:32:24] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:32:27] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:32:30] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:32:33] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:32:36] WARNI

[13:34:05] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:34:06] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:34:08] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:34:10] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:34:12] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:34:14] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:34:15] WARNI

[13:36:15] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:36:15] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:36:16] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:36:16] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:36:17] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:36:17] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:36:18] WARNI

[13:37:25] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:37:27] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:37:30] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:37:32] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:37:35] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:37:38] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:37:41] WARNI

...

[16:33:51] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:33:53] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:33:54] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:33:55] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:33:57] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:33:59] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:34:01] WARNI

[16:37:21] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:37:29] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:37:38] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:37:47] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:37:56] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:38:05] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:38:17] WARNI

[16:42:34] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:42:39] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:42:44] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:42:50] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:42:56] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:43:01] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:43:05] WARNI

[16:48:42] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1, gamma=0,
                                    importance_type='gain', learning_rate=0.1,
                                    max_delta_step=0, max_depth=3,
                                    min_child_weight=1, missing=None,
                                    n_estimators=100, n_jobs=1, nthread=None,
                                    objective='reg:linear', random_state=0,
                                    reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, seed=None, silent=None,
                                    subsample=1, verbosity=1),
             iid='warn', n_jobs=None,
             param_grid={'alpha': [0, 1, 10, 30, 50],
                         'gamma': [0, 1, 10, 30, 50],
              

In [25]:
score = pd.DataFrame(cv_bst.cv_results_ ).sort_values(by='mean_test_score', ascending = False)
score = score.drop(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'params', 
                    'split0_test_score', 'split1_test_score', 'split2_test_score', 'std_test_score', 
                    'rank_test_score'], axis=1)

In [49]:
score['alpha'] = score['param_alpha']
score['gamma'] = score['param_gamma']
score['learning_rate'] = score['param_learning_rate']
score['max_depth'] = score['param_max_depth']
score['n_estimators'] = score['param_n_estimators']
score = score.drop(['param_alpha', 'param_gamma', 'param_learning_rate', 'param_max_depth', 'param_n_estimators'], axis=1)
score

,mean_test_score,alpha,gamma,learning_rate,max_depth,n_estimators
728,0.553123,10,50,0.1,2,60
278,0.553123,1,0,0.1,2,60
528,0.553123,10,0,0.1,2,60
978,0.553123,30,50,0.1,2,60
1078,0.553123,50,1,0.1,2,60
...,...,...,...,...,...,...
820,0.419312,30,1,0.05,12,30
1070,0.419312,50,1,0.05,12,30
570,0.419312,10,1,0.05,12,30
70,0.419312,0,1,0.05,12,30


In [47]:
bst = xgb.XGBRegressor(alpha=10, max_depth=2, n_estimators=80, learning_rate=0.1, gamma=50)
bst.fit(X_train, y_train)
y_pred = bst.predict(X_test)
print(mean_squared_error(y_test, y_pred), r2_score(y_test, y_pred))

[17:06:24] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
32.56581451978647 0.6213678709530915
